<a href="https://colab.research.google.com/github/jetsonmom/halla_ai/blob/main/yolov10_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

커서가 알려준 내용

In [ ]:
!pip install ultralytics opencv-python numpy torch torchvision
!pip install numpy torch torchvision

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

def run_object_detection():
    # YOLOv10x 모델 로드 (더 큰 모델, 더 정확한 감지)
    model = YOLO('yolov10x.pt')

    # 웹캠 초기화
    cap = cv2.VideoCapture(0)

    # FPS 계산을 위한 변수들
    prev_frame_time = 0
    new_frame_time = 0
    font = cv2.FONT_HERSHEY_SIMPLEX

    while True:
        # 프레임 읽기
        ret, frame = cap.read()
        if not ret:
            break

        # FPS 계산
        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)

        # YOLOv10x로 객체 감지
        results = model(frame, conf=0.25)  # confidence threshold 0.25

        # 결과 시각화
        annotated_frame = results[0].plot()

        # FPS 표시
        cv2.putText(annotated_frame, f'FPS: {fps}', (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)

        # 화면에 표시
        cv2.imshow('YOLOv10x Object Detection', annotated_frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 리소스 해제
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # 여기에 실행할 코드를 넣어주세요
    print("프로그램 시작")

In [ ]:
!pip uninstall pytube
!pip install pytube

In [ ]:
!pip install --upgrade yt-dlp

In [ ]:
# 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
import yt_dlp
from google.colab.patches import cv2_imshow  # 코랩용 이미지 표시 함수

def download_youtube_video(url):
    try:
        print("비디오 다운로드 중...")
        ydl_opts = {
            'format': 'best',
            'outtmpl': 'video.mp4',
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        print("다운로드 완료: video.mp4")
        return 'video.mp4'
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return None

def process_video(video_path):
    # YOLOv8x 모델 로드
    model = YOLO('yolov8x.pt')

    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)

    # 비디오 정보 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 결과 비디오 저장을 위한 설정
    output_path = 'output_detection.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 진행률 표시
        frame_count += 1
        progress = (frame_count / total_frames) * 100
        print(f"처리 중... {progress:.1f}%")

        # 객체 감지
        results = model(frame, conf=0.25)

        # 결과 시각화
        annotated_frame = results[0].plot()

        # FPS 표시
        cv2.putText(annotated_frame, f'Frame: {frame_count}/{total_frames}',
                   (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 2)

        # 결과 프레임 저장
        out.write(annotated_frame)

        # 코랩에서 미리보기 표시 (cv2.imshow 대신 cv2_imshow 사용)
        cv2_imshow(annotated_frame)

        # 잠시 대기 (프레임 표시 시간 조절)
        time.sleep(0.1)

    # 리소스 해제
    cap.release()
    out.release()

    return output_path

def main():
    # YouTube Shorts URL
    url = "https://www.youtube.com/shorts/wopMTCh-nSo"

    print("프로그램 시작")

    # 비디오 다운로드
    video_path = download_youtube_video(url)
    if video_path:
        # 비디오 처리
        output_path = process_video(video_path)
        print(f"처리 완료! 결과 파일: {output_path}")

        # 원본 비디오 삭제 (선택사항)
        os.remove(video_path)
    else:
        print("비디오 다운로드 실패")

if __name__ == "__main__":
    main()

위의 코드를 수정해서 사람수를 카운트 자동차수를 카운트하게 만들어본다.

In [ ]:
# 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
import yt_dlp
from google.colab.patches import cv2_imshow
from collections import defaultdict

def download_youtube_video(url):
    try:
        print("비디오 다운로드 중...")
        ydl_opts = {
            'format': 'best',
            'outtmpl': 'video.mp4',
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        print("다운로드 완료: video.mp4")
        return 'video.mp4'
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return None

def count_objects(results):
    counts = defaultdict(int)
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            if conf > 0.25:
                class_name = result.names[cls]
                counts[class_name] += 1
    return counts

def process_video(video_path):
    # YOLOv8x 모델 로드
    model = YOLO('yolov8x.pt')

    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)

    # 비디오 정보 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 결과 비디오 저장을 위한 설정
    output_path = 'output_detection.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    max_counts = defaultdict(int)

    # 프레임 스킵 설정 (처리할 프레임 간격)
    frame_skip = 2  # 2프레임마다 1프레임만 처리

    print(f"총 프레임 수: {total_frames}")
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 스킵
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        # 진행률 표시
        frame_count += 1
        progress = (frame_count / total_frames) * 100
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0

        print(f"처리 중... {progress:.1f}% (FPS: {fps:.1f})")

        # 객체 감지 (더 작은 이미지 크기 사용)
        results = model(frame, conf=0.25, imgsz=640)

        # 현재 프레임의 객체 카운트
        current_counts = count_objects(results)

        # 최대 카운트 업데이트
        for obj, count in current_counts.items():
            max_counts[obj] = max(max_counts[obj], count)

        # 결과 시각화
        annotated_frame = results[0].plot()

        # 카운트 정보 표시
        y_offset = 70
        for obj, count in current_counts.items():
            text = f'{obj}: {count}'
            cv2.putText(annotated_frame, text, (7, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 2)
            y_offset += 40

        # FPS 표시
        cv2.putText(annotated_frame, f'FPS: {fps:.1f}',
                   (7, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 0), 2)

        # 결과 프레임 저장
        out.write(annotated_frame)

        # 코랩에서 미리보기 표시
        cv2_imshow(annotated_frame)

        # 잠시 대기 (시간 단축)
        time.sleep(0.05)

    # 리소스 해제
    cap.release()
    out.release()

    # 최종 카운트 결과 출력
    print("\n=== 최종 카운트 결과 ===")
    for obj, count in max_counts.items():
        print(f"{obj}: {count}")

    total_time = time.time() - start_time
    print(f"\n총 처리 시간: {total_time:.2f}초")
    print(f"평균 FPS: {frame_count/total_time:.1f}")

    return output_path

def main():
    # YouTube Shorts URL
    url = "https://www.youtube.com/shorts/wopMTCh-nSo"

    print("프로그램 시작")

    # 비디오 다운로드
    video_path = download_youtube_video(url)
    if video_path:
        # 비디오 처리
        output_path = process_video(video_path)
        print(f"\n처리 완료! 결과 파일: {output_path}")

        # 원본 비디오 삭제 (선택사항)
        os.remove(video_path)
    else:
        print("비디오 다운로드 실패")

if __name__ == "__main__":
    main()